%md
## PONTIFICIA UNIVERSIDAD JAVERIANA ##

Materia: Procesamieno de datos a gran escala

Limpieza de datos Nivel educativo por municipio

In [1]:
import findspark
findspark.init()

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
%matplotlib inline
from matplotlib import rcParams
import squarify
from sklearn.metrics import roc_curve, auc

#Biblioteca PySpark
import warnings
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import input_file_name, mean, col, split, regexp_extract, when, lit, isnan, count, udf
from pyspark import SparkFiles
from pyspark.sql.types import *

#Bioblioteca ML para PySpark
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, sum
#Importar el SO
from time import time
from pyspark.conf import SparkConf
import os

# Conexión a Spark

In [2]:
SPARK_MASTER_URL = os.getenv("SPARK_MASTER_URL", "spark://10.43.103.125:7077")
configura = SparkConf()
configura.setMaster(SPARK_MASTER_URL)
configura.set('spark.local.dir', '/almacen')
configura.setAppName("PrimerSparkGonzalez")
spark = SparkSession.builder.config(conf=configura).getOrCreate()
SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)
# spark = spark.sparkContext

### Revisar si El servicio esta vivo


spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/10 22:30:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/10 22:30:35 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
url = "https://raw.githubusercontent.com/Kahsma/Proyecto_PDGE/refs/heads/main/Datos/Niveldeeducacion.csv"

spark.sparkContext.addFile(url)


In [4]:
NivelEducación = spark.read.csv("file://" + SparkFiles.get("Niveldeeducacion.csv"), header = True , inferSchema = True )

In [5]:
NivelEducación.show(3)

24/11/10 22:30:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----------------+-------------+-------------------+------------+----------+-------------+--------------+-----------------------+--------------+-------------------------+-----------------------+-------------------------+--------------------+---------------+--------------------------+------------------------+--------------------------+---------------------+------------------------+---------------------------+---------+--------------------+------------------+--------------------+---------------+----------+---------------------+-------------------+---------------------+----------------+-----------+----------------------+--------------------+----------------------+-----------------+----------+---------------------+-------------------+---------------------+----------------+
| AÑO|CÓDIGO_MUNICIPIO|    MUNICIPIO|CÓDIGO_DEPARTAMENTO|DEPARTAMENTO|CÓDIGO_ETC|          ETC|POBLACIÓN_5_16|TASA_MATRICULACIÓN_5_16|COBERTURA_NETA|COBERTURA_NETA_TRANSICIÓN|COBERTURA_NETA_PRIMARIA|COBERTURA_NETA_S

In [6]:
# Calcular los valores nulos para cada columna
conteo_nulos = NivelEducación.select([sum(col(c).isNull().cast("int")).alias(c) for c in NivelEducación.columns])

# Mostrar el resultado
conteo_nulos.show()


+---+----------------+---------+-------------------+------------+----------+---+--------------+-----------------------+--------------+-------------------------+-----------------------+-------------------------+--------------------+---------------+--------------------------+------------------------+--------------------------+---------------------+------------------------+---------------------------+---------+--------------------+------------------+--------------------+---------------+----------+---------------------+-------------------+---------------------+----------------+-----------+----------------------+--------------------+----------------------+-----------------+----------+---------------------+-------------------+---------------------+----------------+
|AÑO|CÓDIGO_MUNICIPIO|MUNICIPIO|CÓDIGO_DEPARTAMENTO|DEPARTAMENTO|CÓDIGO_ETC|ETC|POBLACIÓN_5_16|TASA_MATRICULACIÓN_5_16|COBERTURA_NETA|COBERTURA_NETA_TRANSICIÓN|COBERTURA_NETA_PRIMARIA|COBERTURA_NETA_SECUNDARIA|COBERTURA_NETA_MEDIA

In [7]:
# Eliminar espacios en los nombres de las columnas
for nombre_columna in NivelEducación.columns:
    NivelEducación = NivelEducación.withColumnRenamed(nombre_columna, nombre_columna.strip())

# Mostrar el DataFrame para confirmar los cambios
NivelEducación.show(3)


+----+----------------+-------------+-------------------+------------+----------+-------------+--------------+-----------------------+--------------+-------------------------+-----------------------+-------------------------+--------------------+---------------+--------------------------+------------------------+--------------------------+---------------------+------------------------+---------------------------+---------+--------------------+------------------+--------------------+---------------+----------+---------------------+-------------------+---------------------+----------------+-----------+----------------------+--------------------+----------------------+-----------------+----------+---------------------+-------------------+---------------------+----------------+
| AÑO|CÓDIGO_MUNICIPIO|    MUNICIPIO|CÓDIGO_DEPARTAMENTO|DEPARTAMENTO|CÓDIGO_ETC|          ETC|POBLACIÓN_5_16|TASA_MATRICULACIÓN_5_16|COBERTURA_NETA|COBERTURA_NETA_TRANSICIÓN|COBERTURA_NETA_PRIMARIA|COBERTURA_NETA_S

In [8]:
# Seleccionar solo las columnas especificadas
columnas_a_mantener = ["CÓDIGO_MUNICIPIO", "TASA_MATRICULACIÓN_5_16", "COBERTURA_BRUTA", "SEDES_CONECTADAS_A_INTERNET"]
NivelEducación = NivelEducación.select(*columnas_a_mantener)

# Mostrar el resultado para confirmar
NivelEducación.show(10)


+----------------+-----------------------+---------------+---------------------------+
|CÓDIGO_MUNICIPIO|TASA_MATRICULACIÓN_5_16|COBERTURA_BRUTA|SEDES_CONECTADAS_A_INTERNET|
+----------------+-----------------------+---------------+---------------------------+
|           99773|                  49.85|          56.18|                       NULL|
|           99624|                  83.87|           93.1|                       NULL|
|           99524|                  95.94|         105.45|                       NULL|
|           99001|                 144.52|          158.5|                       NULL|
|           97889|                  47.95|          52.05|                       NULL|
|           97777|                    0.0|              0|                       NULL|
|           97666|                   22.6|          23.34|                       NULL|
|           97511|                  54.29|          71.39|                       NULL|
|           97161|                   51.3| 

In [9]:
# Eliminar filas con cualquier valor nulo
NivelEducación = NivelEducación.dropna()

# Mostrar el resultado para confirmar
NivelEducación.show(10)


+----------------+-----------------------+---------------+---------------------------+
|CÓDIGO_MUNICIPIO|TASA_MATRICULACIÓN_5_16|COBERTURA_BRUTA|SEDES_CONECTADAS_A_INTERNET|
+----------------+-----------------------+---------------+---------------------------+
|           85015|                   58.4|          65.23|                      11.11|
|           85125|                   88.9|         106.08|                      16.67|
|           85136|                   70.9|          76.63|                       12.5|
|           85139|                  122.5|         139.05|                      29.03|
|           85162|                   91.9|         108.29|                      84.62|
|           85225|                   78.2|          89.06|                       9.52|
|           85230|                  114.5|          132.5|                       20.0|
|           85250|                  121.9|         145.07|                       9.33|
|           85263|                  124.8| 

In [10]:
# Calcular valores nulos para cada columna
conteo_nulos = NivelEducación.select([sum(col(c).isNull().cast("int")).alias(c) for c in NivelEducación.columns])

# Mostrar el resultado
conteo_nulos.show()


+----------------+-----------------------+---------------+---------------------------+
|CÓDIGO_MUNICIPIO|TASA_MATRICULACIÓN_5_16|COBERTURA_BRUTA|SEDES_CONECTADAS_A_INTERNET|
+----------------+-----------------------+---------------+---------------------------+
|               0|                      0|              0|                          0|
+----------------+-----------------------+---------------+---------------------------+



In [11]:
# Obtener el conteo total de registros
total_registros = NivelEducación.count()

# Mostrar el resultado
print(f"Número total de registros: {total_registros}")


Número total de registros: 7667


In [12]:
# Contar el número total de registros antes de eliminar duplicados
conteo_inicial = NivelEducación.count()

# Eliminar duplicados basados en 'CÓDIGO_MUNICIPIO' y mantener la primera ocurrencia
NivelEducación = NivelEducación.dropDuplicates(subset=["CÓDIGO_MUNICIPIO"])

# Contar el número total de registros después de eliminar duplicados
conteo_final = NivelEducación.count()

# Calcular el número de duplicados eliminados
duplicados_eliminados = conteo_inicial - conteo_final

# Mostrar el resultado
print(f"Número de registros duplicados eliminados: {duplicados_eliminados}")
print(f"Total de registros después de eliminar duplicados: {conteo_final}")


Número de registros duplicados eliminados: 6549
Total de registros después de eliminar duplicados: 1118


In [13]:
# Definir la ruta de salida para el archivo CSV
ruta_salida = "Proyecto/"  # Reemplazar con la ruta de tu directorio deseado

# Escribir el DataFrame en un archivo CSV
NivelEducación.write.csv(ruta_salida, header=True, mode="overwrite")
